In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import math

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

In [12]:
#df = pd.read_excel('leaders_bert.xlsx')
df = pd.read_excel('/content/drive/MyDrive/MSc/public_topics_bertopic.xlsx')
len(df)

16736

In [13]:
df['bert_text'] = df['bert_text'].str.lower()

# BERT

- https://huggingface.co/blog/sentiment-analysis-python
- https://huggingface.co/docs/transformers/main_classes/pipelines

In [16]:
tokenizer = AutoTokenizer.from_pretrained('tartuNLP/EstBERT')

In [17]:
# load in the pretrained model
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/MSc/MAKATÖÖ/bertSentiment/',local_files_only=True)

In [18]:
 sentiment_model = pipeline(task = 'sentiment-analysis', model = model, tokenizer = tokenizer)

In [19]:
%%time

import textwrap


sentiment_bert = []

for text in df['bert_text']:
    if len(text) > 512:
        split_into = math.ceil(len(text) / 512)
        split_len = math.ceil(len(text) / split_into)
        text_splits = textwrap.wrap(text, split_len)
        results = 0
        for split in text_splits:
            split_sentiment = sentiment_model([split])
            result_number = split_sentiment[0]['label'].split("_")[1]
            results += int(result_number)
            
        final_label = 'LABEL_' + str(round(results / len(text_splits)))
        sentiment_bert.append(final_label)
    else:
        sentiment = sentiment_model([text])
        sentiment_bert.append(sentiment[0]['label'])

CPU times: user 47min 53s, sys: 4.35 s, total: 47min 57s
Wall time: 48min 24s


In [20]:
from collections import Counter
Counter(sentiment_bert)

Counter({'LABEL_0': 11517, 'LABEL_2': 2282, 'LABEL_1': 2937})

In [ ]:
# add sentiments to df
df['sentiment_bert'] = sentiment_bert
df

In [22]:
# save file
df.to_excel('public_sentiment_bert.xlsx', index = False)